<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U06/ANN06_resnet_TEMPLATE_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# ANN U06: CNN für Computer Vision
### Imports
Hier werden alle notwendigen Bibliotheken für das Training eines neuronalen Netzes mit PyTorch Lightning importiert.

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import seaborn as sns
import pytorch_lightning as pl
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torchviz import make_dot
from pytorch_lightning import Trainer
from torchvision import transforms, datasets, models

from torchmetrics.functional import accuracy
from sklearn.metrics import confusion_matrix, classification_report


## Aufgabe 1: ResNet – Training und Evaluation

## (a) Allgemeine Einstellungen und Konstanten

In [ ]:
# Diese Werte werden als Standard genutzt, falls die automatische Berechnung deaktiviert ist.
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2470, 0.2435, 0.2616)
BATCH_SIZE = 128
NUM_WORKERS = 4


### Dictionary: Übersetzung der one-hot-codierten Vektoren in reale Labels

In [ ]:
classes = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck",
}


### Hilfsfunktion: Unnormalisieren der Bilder für Visualisierungen

In [ ]:
def unnormalize(img, mean, std):
    # Erstelle eine Kopie des Bildes, um die Originaldaten nicht zu verändern
    img = img.clone().detach()
    # Denormalisiere das Bild, indem die Mittelwerte hinzugefügt und durch die Standardabweichungen geteilt werden
    for t, m, s in zip(img, mean, std):
        t.mul_(s).add_(m)
    return img


### DataModule: CIFAR-10 Vorbereitung und DataLoader mit automatischer Berechnung von Mean und Std

In [ ]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, data_dir="./data", batch_size=BATCH_SIZE, auto_normalize=True):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.auto_normalize = auto_normalize

        # Platzhalter für die Normalisierungswerte
        self.mean = None
        self.std = None

        # Vorläufige Transformation: Nur in Tensor umwandeln
        self.base_transform = transforms.ToTensor()

        # Die finale Transformationskette wird in setup() definiert, nachdem ggf. Mean/Std berechnet wurden.
        self.transform = None

    def prepare_data(self):
        # Datensatz herunterladen, falls noch nicht vorhanden
        datasets.CIFAR10(self.data_dir, train=True, download=True)
        datasets.CIFAR10(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Erstelle den Trainingsdatensatz ohne Normalisierung, um die Werte zu berechnen
        train_dataset = datasets.CIFAR10(
            self.data_dir, train=True, transform=self.base_transform
        )

        if self.auto_normalize:
            loader = torch.utils.data.DataLoader(
                train_dataset, batch_size=100, shuffle=False, num_workers=NUM_WORKERS
            )
            mean = torch.zeros(3)
            std = torch.zeros(3)
            nb_samples = 0
            for data, _ in loader:
                batch_samples = data.size(0)
                data = data.view(batch_samples, data.size(1), -1)
                mean += data.mean(2).sum(0)
                std += data.std(2).sum(0)
                nb_samples += batch_samples
            self.mean = (mean / nb_samples).tolist()
            self.std = (std / nb_samples).tolist()
        else:
            self.mean = CIFAR10_MEAN
            self.std = CIFAR10_STD

        # Jetzt die finale Transformation definieren (inklusive Normalisierung)
        self.transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize(self.mean, self.std)]
        )

        # Erstelle die Datensätze mit der kompletten Transformationskette
        self.cifar10_train = datasets.CIFAR10(
            self.data_dir, train=True, transform=self.transform
        )
        # Hinweis: Hier wird der Trainingsdatensatz auch als Validierungsdatensatz genutzt.
        self.cifar10_val = datasets.CIFAR10(
            self.data_dir, train=True, transform=self.transform
        )
        self.cifar10_test = datasets.CIFAR10(
            self.data_dir, train=False, transform=self.transform
        )

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.cifar10_train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=NUM_WORKERS,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.cifar10_val,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=NUM_WORKERS,
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.cifar10_test,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=NUM_WORKERS,
        )


### LightningModule: ResNet-Modell für CIFAR-10

In [ ]:
class ResNetLightning(pl.LightningModule):
    def __init__(self, num_classes=10, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters()
        # Verwende ResNet18, angepasst für CIFAR-10
        self.model = models.resnet18(weights=None)
        # Anpassung der ersten Convolution: CIFAR-10 Bilder haben 32x32 Pixel
        self.model.conv1 = nn.Conv2d(
            3, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.model.maxpool = nn.Identity()  # Entferne die MaxPooling-Schicht
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)

        self.criterion = nn.CrossEntropyLoss()
        self.test_preds = []
        self.test_targets = []

    def forward(self, x):
        # Vorwärtsdurchlauf des Modells
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # Trainingsschritt
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = accuracy(F.softmax(logits, dim=1), y, task="multiclass", num_classes=10)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Validierungsschritt
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = accuracy(F.softmax(logits, dim=1), y, task="multiclass", num_classes=10)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # Testschritt
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.test_preds.append(preds.cpu())
        self.test_targets.append(y.cpu())
        acc = accuracy(F.softmax(logits, dim=1), y, task="multiclass", num_classes=10)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss

    def on_test_epoch_end(self):
        # Am Ende der Test-Epoche: Konfusionsmatrix und Klassifikationsreport anzeigen
        preds = torch.cat(self.test_preds)
        targets = torch.cat(self.test_targets)
        cm = confusion_matrix(targets, preds)
        print("Konfusionsmatrix:")
        plt.figure(figsize=(8, 6))
        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            xticklabels=classes,
            yticklabels=classes,
            cmap="Blues",
        )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix")
        plt.show()

        print("\nKlassifikationsreport:")
        print(classification_report(targets, preds, target_names=list(classes.values())))

    def configure_optimizers(self):
        # Optimierer konfigurieren
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer


### Visualisierung der Modellarchitektur

In [ ]:
def visualize_model(model):
    dummy_input = torch.randn(1, 3, 32, 32)
    out = model(dummy_input)
    dot = make_dot(out, params=dict(model.named_parameters()))
    dot.format = "png"
    dot.render("resnet_architecture")
    img = plt.imread("resnet_architecture.png")
    plt.figure(figsize=(40, 40))
    plt.imshow(img)
    plt.axis("off")
    plt.title("ResNet Architektur")
    plt.show()


### Darstellung der ersten 25 Testbilder

In [ ]:
def plot_test_images(data_module):
    test_loader = data_module.test_dataloader()
    images, labels = next(iter(test_loader))
    images = images[:25]
    labels = labels[:25]

    fig, axes = plt.subplots(5, 5, figsize=(10, 10))
    axes = axes.flatten()
    for img, label, ax in zip(images, labels, axes):
        # Nutze hier die automatisch berechneten Mean/Std zur Unnormalisierung
        img = unnormalize(img, data_module.mean, data_module.std)
        img = img.permute(1, 2, 0).numpy()
        ax.imshow(np.clip(img, 0, 1))
        ax.set_title(classes[label.item()])
        ax.axis("off")
    plt.tight_layout()
    plt.show()


### Darstellung der Softmax-Ausgaben für die ersten 6 Bilder

In [ ]:
def plot_softmax_outputs(model, data_module):
    test_loader = data_module.test_dataloader()
    images, labels = next(iter(test_loader))
    images = images[:6]
    labels = labels[:6]

    model.eval()
    with torch.no_grad():
        logits = model(images)
        probs = F.softmax(logits, dim=1).cpu().numpy()

    fig, axes = plt.subplots(6, 2, figsize=(20, 24))
    for i in range(6):
        # Linke Spalte: Bildanzeige
        img_disp = (
            unnormalize(images[i], data_module.mean, data_module.std)
            .permute(1, 2, 0)
            .numpy()
        )
        axes[i, 0].imshow(np.clip(img_disp, 0, 1))
        axes[i, 0].set_title(f"True: {classes[labels[i].item()]}")
        axes[i, 0].axis("off")
        # Rechte Spalte: Balkendiagramm der Softmax-Ausgabe
        axes[i, 1].bar(range(len(probs[i])), probs[i])
        axes[i, 1].set_yscale("log")
        axes[i, 1].set_xticks(range(len(probs[i])))
        axes[i, 1].set_xticklabels(list(classes.values()), rotation=45)
        axes[i, 1].set_title("Softmax-Ausgabe")
        axes[i, 1].set_xlabel("Klassen")
        axes[i, 1].set_ylabel("Wahrscheinlichkeit (log)")
    plt.tight_layout()
    plt.show()


### Training, Evaluation und zusätzliche Visualisierungen

In [ ]:
class TrainLossHistory(pl.Callback):
    def __init__(self):
        self.train_losses = []

    def on_train_epoch_end(self, trainer, pl_module):
        loss = trainer.callback_metrics.get("train_loss")
        if loss is not None:
            self.train_losses.append(loss.cpu().detach().item())


print("Aufgabe 1: ResNet – Training und Evaluation")
pl.seed_everything(42)

# DataModule und Modell initialisieren
# Hier wird die automatische Berechnung der Normalisierungswerte aktiviert.
data_module = CIFAR10DataModule(auto_normalize=True)
model = ResNetLightning()


## (b) Visualisierung der Modellarchitektur

In [ ]:
# Visualisierung der Modellarchitektur
visualize_model(model)


In [ ]:
# Geräteauswahl: GPU, MPS (Apple) oder CPU
if torch.cuda.is_available():
    accelerator = "gpu"
    devices = 1
    print("Verwende GPU für das Training.")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    accelerator = "mps"
    devices = 1
    print("Verwende MPS (Apple Silicon) für das Training.")
else:
    accelerator = "cpu"
    devices = None
    print("Verwende CPU für das Training.")

loss_history = TrainLossHistory()
trainer = Trainer(
    max_epochs=20, accelerator=accelerator, devices=devices, callbacks=[loss_history]
)


## (c) Training des ResNet

In [ ]:
# Training des ResNet
trainer.fit(model, datamodule=data_module)


## (d-e) Konfusionsmatrix, Klassifikationsreport, Lernkurve

In [ ]:
# Evaluation: Testen, Konfusionsmatrix & Klassifikationsreport
trainer.test(model, datamodule=data_module)

# Plot Trainings-Losskurve
plt.figure(figsize=(8, 6))
plt.plot(loss_history.train_losses, label="Train Loss", marker="o")
plt.xlabel("Epoche")
plt.ylabel("Loss")
plt.title("Trainings-Losskurve")
plt.legend()
plt.show()


## (f) Darstellung der ersten 25 Testbilder 

In [ ]:
# Darstellung der ersten 25 Testbilder
plot_test_images(data_module)


## (g) Softmax Ausgaben

In [ ]:
# Darstellung der Softmax-Ausgaben für die ersten 6 Bilder
plot_softmax_outputs(model, data_module)


Die Bearbeitung der Aufgabe 2 bietet eine hervorragende Gelegenheit, PyTorch Lightning praktisch auszuprobieren und ein tieferes Verständnis für dessen Funktionsweise zu entwickeln. Durch das Testen verschiedener Ansätze können wertvolle Erfahrungen gesammelt werden, die beim effizienten Einsatz dieser leistungsstarken Bibliothek helfen.